<a href="https://colab.research.google.com/github/K100NUNES/Ticket-Monitor/blob/main/monitor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install slack-sdk
#!pip install selenium
#!pip install jupyter
#!pip install Pillow

In [2]:
from selenium import webdriver
from PIL import Image, ImageChops
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import os
import datetime
import time
import io

# Defina suas credenciais do Slack
slack_token = os.environ.get("SLACK_TOKEN")
channel_id = os.environ.get("SLACK_CHANNEL")
bot_name = 'ticket_monitor'

# Configurações do ChromeDriver para evitar travamentos
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')

# Inicializar o driver do Chrome
driver = webdriver.Chrome(options=chrome_options)

# Abrir a página que você deseja monitorar
url = "https://spfc.totalacesso.com/"
driver.get(url)

# Aumentar a largura e a altura da janela do navegador para capturar uma área maior
driver.set_window_size(1920, 1080)  # Por exemplo, definindo para 1920x1080 pixels

# Inicializar o cliente Slack
slack_client = WebClient(token=slack_token)

# Definir o arquivo de log
log_file = 'monitoring_log.txt'

def log_status(status):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, 'a') as f:
        f.write(f"[{timestamp}] {status}\n")

# Verificar se o screenshot inicial já foi enviado
screenshot_inicial_enviado = os.path.isfile('screenshot_inicial_enviado.txt')

# Lê o valor de primeira_execucao de um arquivo
primeira_execucao = not screenshot_inicial_enviado

# Capturar o screenshot inicial apenas uma vez na primeira execução
if primeira_execucao:
    driver.save_screenshot('screenshot_inicial.png')
    initial_image = Image.open('screenshot_inicial.png')

    try:
        with open('screenshot_inicial.png', 'rb') as f:
            response = slack_client.files_upload(
                channels=channel_id,
                file=io.BytesIO(f.read()),  # Enviar o screenshot inicial
                filename="screenshot_inicial.png",
                title="Registro Inicial da Captura",
                initial_comment="Registro inicial da captura.",
                username=bot_name  # Defina o nome do bot aqui
            )
        log_status("Screenshot inicial enviado para o Slack.")

        # Marcar que o screenshot inicial foi enviado
        with open('screenshot_inicial_enviado.txt', 'w') as f:
            f.write('Enviado')
    except SlackApiError as e:
        log_status(f"Erro ao enviar screenshot inicial para o Slack: {e.response['error']}")

while True:
    # Capturar um novo screenshot após as ações de monitoramento
    driver.save_screenshot('screenshot_apos_monitoramento.png')

    # Comparar o screenshot atual com o screenshot inicial
    new_image = Image.open('screenshot_apos_monitoramento.png')
    diff = ImageChops.difference(initial_image, new_image)

    # Se não for a primeira execução, verificar se há diferenças
    # Se houver diferenças significativas, enviar notificação para o Slack
    if not primeira_execucao and diff.getbbox():
        # Enviar a notificação para o Slack com o novo screenshot
        try:
            with open('screenshot_apos_monitoramento.png', 'rb') as f:
                response = slack_client.files_upload(
                    channels=channel_id,
                    file=io.BytesIO(f.read()),  # Enviar apenas a diferença detectada
                    filename="screenshot_apos_monitoramento.png",
                    title="Diferença Detectada",
                    initial_comment="Uma diferença significativa foi detectada na página.",
                    username=bot_name  # Defina o nome do bot aqui
                )
            log_status("Diferença detectada. Notificação enviada para o Slack.")
        except SlackApiError as e:
            log_status(f"Erro ao enviar screenshot de diferença para o Slack: {e.response['error']}")

        # Atualizar o screenshot inicial
        initial_image = new_image

    # Se não for a primeira execução e não houver diferenças,
    # envie uma mensagem informando que não houve alterações e encerre o programa
    if not primeira_execucao and not diff.getbbox():
        log_status("Nenhuma alteração significativa foi detectada na página.")
        try:
            slack_client.chat_postMessage(
                channel=channel_id,
                text="Nenhuma alteração significativa foi detectada na página."
            )
        except SlackApiError as e:
            log_status(f"Erro ao enviar mensagem de status para o Slack: {e.response['error']}")

        # Fechar o driver do Chrome e encerrar o programa
        driver.quit()
        break

    # Se for a primeira execução, atualize o valor de primeira_execucao
    if primeira_execucao:
        primeira_execucao = False



/opt/hostedtoolcache/Python/3.11.4/x64/lib/python3.11/site-packages/slack_sdk/web/internal_utils.py:417: UserWarning: client.files_upload() may cause some issues like timeouts for relatively large files. Our latest recommendation is to use client.files_upload_v2(), which is mostly compatible and much stabler, instead.
  warnings.warn(message)
